In [ ]:
import torch
import torch.nn as nn

In [3]:
chemin_model="results/finetuned_models/ftd_dirichlet_thresh0.6.pth"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
model = torch.load(chemin_model, map_location=device)
model.eval()
b = model

C:\Users\skouz\AppData\Local\Temp\ipykernel_16888\55063412.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(chemin_model, map_location=device)


In [6]:
class VitONNXWrapper(nn.Module):
    def __init__(self, vit_model):
        super().__init__()
        self.vit_model = vit_model

    def forward(self, x):
        out = self.vit_model(x)
        return out.number_logits

In [7]:
wrapped_model = VitONNXWrapper(b).to(device)
wrapped_model.eval()
a = wrapped_model

In [8]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)

In [9]:
torch.onnx.export(
    wrapped_model,
    dummy_input,
    "vit_model.onnx",
    input_names=["input"],
    output_names=["number_logits"],  # name logique
    dynamic_axes={"input": {0: "batch_size"}, "number_logits": {0: "batch_size"}},
    opset_version=14,
    do_constant_folding=True,
)

c:\Users\skouz\anaconda3\envs\jersey\lib\site-packages\transformers\models\vit\modeling_vit.py:154: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:
c:\Users\skouz\anaconda3\envs\jersey\lib\site-packages\transformers\models\vit\modeling_vit.py:160: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if height != self.image_size[0] or width != self.image_size[1]:
